## Ad Auctions for LLMs via Retrieval Augmented Genearation

Here is the outline of the experiments we plan to run on this notebook:

1. Given a query $x$, we generate fictional advertisers $\text{ad}_1, \text{ad}_2, ..., \text{ad}_n$ with their ads.
2. We sample bids $b_1, b_2, ..., b_n$ from a lognormal distribution (mean and std of this should be adjusted w.r.t. values of $q_i$)
3. We run different mechanism for this LLM query/ads configuration, *i.e.*, $(x, \{ad_i\}_{i=1}^{n}, \{b_i\}_{i=1}^{n}$.

    + single-allocation segment auction with replacement
    + single-allocation segment auction without replacement
    + multi-allocation greedy mechanism
    + naive (i) mechanism where $y = y_{\text{orig}} + \text{ad}_1 + \text{ad}_2 + ... + \text{ad}_k$
    + naive (ii) mechanism where we run 2nd price auction in each round

4. We run $N$ times each of the above mechanisms as they are randomized, we report the expectation of the following metrics:
    + social welfare (sum of $v_{i_t} q_{i_t}$)
    + revenue (sum of $p_{i_t}$)
    + relevance (sum of $q_{i_t}$)
    + output quality; we think of the following metrics
        + distance of output including ad to the original output -- measures if the output including ad answers the query or not.
        + we ask an off-the-shelf LLM to evaluate the output including ads has too much advertisement or not, probably asking it to rate from $1$ to $10$.
        + we ask an off-the-shelf LLM to evaluate paragraphs are coherent or not, probably asking it to rate from $1$ to $10$.


There are several design choices that we consider:
+ How long each segment should be, we first run with a paragraph.
+ dependant/independent segment auction
+ distribution that $x_i$ (allocation vector) comes from. Since we use RAG-based aggregation function on bids to get the allocation vector, we need to control how this allocation vector looks like, *e.g.*, we were thinking of putting some ratio for $\frac{x_{\max}}{x_{\min}}$.
+ how many segments should we consider? we put $k=3$ in our experiments, this further gives us the results for $k = 1, 2$.
+ design choices for multi-allocation greedy mechanism.

### GPT-4 API Access

In [89]:
from openai import OpenAI


def query_to_chatgpt(prompt):
    chat_completion_det = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        model = "gpt-4-turbo",
        logprobs = False,
        temperature = 1,
        max_tokens = 300
    )
    return chat_completion_det.choices[0].message.content

### Libraries

In [2]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time
import numpy as np
import json
from itertools import combinations_with_replacement

### Models

In [27]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

### Initiating query and advertisers
I borrow samples from the previous experiments we did -- we may need to generate fictional advertisers (ask Sebastien).

In [3]:
prompt = 'How to activate Internationl Roaming?'
advertisers = [
    'AT&T', 'Huawei',
    'Samsung', 'LG',
    'Costco', 'Starbucks',
]

prompt_for_adv = 'Give me a paragraph about {} so that I can use it to advertise this brand.'

### Obtaining $\text{ad}_i$ For All Advertisers

Here, we use the `prompt_for_adv` above to get a paragraph describing each of the advertisers.
Responses will generate a set of advertisements/documents that are used according to RAG to include advertisement within the output. Similar to what we had in paper denoted by: 
$$\{\text{ad}_1, \text{ad}_2, ..., \text{ad}_n\}.$$

In [4]:
%%time

ads = []
for c in tqdm(advertisers):
    ads.append(query_to_chatgpt(prompt_for_adv.format(c)))

100%|██████████| 6/6 [00:50<00:00,  8.43s/it]

CPU times: user 66.1 ms, sys: 9.82 ms, total: 75.9 ms
Wall time: 50.6 s


In [5]:
ads

["Experience seamless connectivity and unparalleled customer service with AT&T, America's leading communications company. With over a century of innovation, AT&T offers cutting-edge solutions in wireless and wireline technology, high-speed internet, and smart entertainment options. Trust in a network that provides widespread, reliable coverage and a wide array of products and services tailored to meet your specific needs, whether for personal use or business solutions. Join millions of satisfied customers who enjoy the latest in technology and entertainment with AT&T, where we're committed to keeping you connected to what matters most.",
 "Huawei stands at the forefront of innovation, delivering cutting-edge technology to enhance your digital experience. Known for its high-quality, reliable products, Huawei offers a diverse range of devices including smartphones, tablets, wearables, and laptops tailored to meet the needs of modern consumers and businesses alike. Whether you're looking 

### Segment-wise RAG-based Output Modification

In [133]:
prompt_to_init_answer = '''
{} please respond to this question for only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph, 4 sentences please.
'''

prompt_to_continue_answer = '''
please answer the question "{}" by continuing your response "{}". please write only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output. Ads should be minimal.
Your output must be 1 paragraph, 4 sentences please.
'''

prompt_to_init_multi_ad_k3 = '''
{} please respond to this question for only 1 paragraph while minimally advertise {}, {}, and {} with these three contexts:  1) "{}" \n 2) "{}" \n 3) "{}".
Make sure that you advertise all of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k3 = '''
please answer the question "{}" by continuing this response "{}". write only 1 paragraph while minimally advertise {}, {}, and {} with these three contexts:  1) "{}" \n 2) "{}" \n 3) "{}".
Make sure that you advertise all of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_init_multi_ad_k2 = '''
{} please respond to this question for only 1 paragraph while minimally advertise {} and {} with these two contexts:  1) "{}" \n 2) "{}".
Make sure that you advertise both of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k2 = '''
please answer the question "{}" by continuing this response "{}". write only 1 paragraph while minimally advertise {} and {} with these two contexts:  1) "{}" \n 2) "{}".
Make sure that you advertise both of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_init_multi_ad_k1 = '''
{} please respond to this question for only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k1 = '''
{} please respond to this question while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output. Ads should be minimal.
Your output must be 1 paragraph.
'''

### Segment-wise Generation
At each segment, we ask LLM to continue its response for prompt $x$ while including ad from the advertiser who has won this round of auction.

In [6]:
def segment_based_RAG_generation_old(prompt: str, advertiser: str, ad: str, curr_y=None):
    if curr_y is None:
        curr_y = query_to_chatgpt(prompt_to_init_answer.format(prompt, advertiser, ad))
    else:
        curr_y += '\n' + query_to_chatgpt(prompt_to_continue_answer.format(prompt, curr_y, advertiser, ad))
    return curr_y

In [185]:
def segment_based_RAG_generation(prompt: str, advertiser: str, ad: str, previous_queries: list, previous_outputs: list):
    # print(type(previous_queries), type(previous_outputs))
    # print(len(previous_queries), len(previous_outputs))
    if len(previous_queries) == 0:
        query = f'''{prompt}\n please respond to this question for only one sentence while also advertise {advertiser} with this context >>
        {ad}
        Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
        Focus on answering the question, there shouldn't be too much advertisment in the output.
        Make sure that response should be one sentence.
        '''
    else:
        query = f'''
        You must continue your answer to my original query.
        Your previous response was
        >> {previous_outputs[-1]}

        And you now should advertise {advertiser}, but without hurting the coherency of the entire document.
        Here's some contexts about {advertiser}

        >>  {ad}

        Make sure that there is one new sentence.
        Write the entire document, which merges your previous response and new paragraph.
        '''
    
    
    messages = []
    for i in range(len(previous_queries)):
        messages.append({
            "role": "user", "content": previous_queries[i]
        })
        messages.append({
            "role": "assistant", "content": previous_outputs[i]
        })
    messages.append({
        "role": "user", "content": query
    })
    
    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        logprobs = False,
        temperature = 1,
        max_tokens = 300,
        messages=messages,)
    
    return query, response.choices[0].message.content

In [7]:
def segment_based_multi_ad_generation(prompt: str, advertisers: list, ads: str, curr_y=None):
    if len(advertisers) == 1:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k1.format(prompt, advertisers[0], ads[0]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k1.format(prompt, curr_y, advertisers[0], ads[0]))
        
    elif len(advertisers) == 2:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, advertisers[0], advertisers[1], ads[0], ads[1]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, curr_y, advertisers[0], advertisers[1], ads[0], ads[1]))
    elif len(advertisers) == 3:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, advertisers[0], advertisers[1], advertisers[2], ads[0], ads[1], ads[2]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, curr_y, advertisers[0], advertisers[1], advertisers[2], ads[0], ads[1], ads[2]))
    else:
        assert 1 == 0
        
    return curr_y    

### Relevance Measure

Below code finds $\text{P}_{\eta}\left[\text{ad}_i\ |\ x\right]$ using embedding space of `SentenceTransformer`.

In [8]:
def rag_based_relevance(x: str, ads: list): 
    x_embedding = model.encode(x)
    ads_embedding = model.encode(ads)
    bf = (1 + util.dot_score(x_embedding, ads_embedding)[0].numpy()) / 2
    return (bf - 0.4) /0.3

### Metrics

We compare different mechanisms w.r.t below metrics. For an auction with $k$ segments, they are defined as follows:
$$\begin{align*}
    \texttt{Revenue}:& \sum_{t=1}^{k} p_{i_t} \\
    \texttt{Social Welfare}:& \sum_{t=1}^{k} v_{i_t} q_{i_t} \\
    \texttt{Relevance}:& \sum_{t=1}^{k} q_{i_t} \\
\end{align*}$$


In [11]:
SOCIAL_WELFARE = 'social_welfare'
REVENUE = 'revenue'
RELEVANCE = 'relevance'
OUTPUT = 'output'
WINNER = 'winner'

In [187]:
def init_metrics():
    return {
        SOCIAL_WELFARE: [],
        REVENUE: [],
        RELEVANCE: [],
        OUTPUT: [],
        WINNER: [],
    }

def update_metrics(metrics: dict, payment: float, value: float, rel: float, output: str, winner: str=''):
    metrics[REVENUE].append(float(payment))
    metrics[SOCIAL_WELFARE].append(float(value * rel))
    metrics[RELEVANCE].append(float(rel))
    metrics[OUTPUT].append(output)
    metrics[WINNER].append(winner)

### Running Auction

We adjust bids $\{b_i\}_{i=1}^{n}$ according to scores coming from RAG, *i.e.*, $q_i$. This is implemented using pages 4 and 5 of the draft.

In [70]:
def randomized_selection(q: np.ndarray, b: np.ndarray):
    adjusted_bids = (q * b) / np.dot(q, b)
    n = b.shape[0] # number of ads
    
    k , u = None, np.random.uniform()
    
    for i in range(n):
        W_i = (np.sum(adjusted_bids[: i]), np.sum(adjusted_bids[: i+1]))
        if W_i[0] <= u <= W_i[1]:
            k = i
    
    all_other_than_k = np.sum(adjusted_bids) - adjusted_bids[k]
    aux_value = (np.sum(adjusted_bids[:k])) / all_other_than_k
    
    if aux_value >= u:
        payment = (np.sum(adjusted_bids[:k]) - u * all_other_than_k) / (u * q[k])
    else:
        payment = (np.sum(adjusted_bids[:k]) - u * all_other_than_k) / ((u - 1) * q[k])
    
    return k, payment
        

In [50]:
def get_allocation_vector(prompt: str, bids: np.ndarray, ads: list, advertisers: list):
    q = rag_based_relevance(prompt, ads, )
    x = q * bids / np.sum(q * bids)
    
    print(f'(q_i, b_i, adv_i)')
    for y in list(zip(q, bids, advertisers)):
        print(f'({y[0]:.2f}, {y[1]:.2f}, {y[2]})')
        
    print(f'allocation vector:\n{x}')
    print(f'x_max / x_min: {np.max(x) / np.min(x)}')

### Mechanisms
+ single-allocation segment auction with replacement
+ single-allocation segment auction without replacement
+ multi-allocation greedy mechanism
+ naive (i) mechanism where $y = y_{\text{orig}} + \text{ad}_1 + \text{ad}_2 + ... + \text{ad}_k$
+ naive (ii) mechanism where we run 2nd price auction in each round

For each of these mechanisms, we need to have a set of bids $\{b_i\}_{i=1}^{n}$ from all advertisers, as well as RAG-based relevancy metric $q_i^{(t)}$. We compute RAG-based relevancy by measuring the similarity of current generated response (or query $x$) with ads (documents). Mechanism is run for $k$ iterations. We store metrics meanwhile that are later reported as properties of different mechanisms.

In [179]:
def single_allocation_with_replacement(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    v = np.copy(bids)
    metrics = init_metrics()
    
    previous_queries, previous_outputs = [], []
    
    if not dependent:
        q = rag_based_relevance(prompt, ads, )
    
    for t in range(num_of_segments):
        if dependent:
            q = rag_based_relevance(prompt + '\n' + curr, ads, )
        
        k, payment = randomized_selection(q, bids)
        query, output = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], previous_queries=previous_queries, previous_outputs=previous_outputs)
        
        previous_queries.append(query)
        previous_outputs.append(output)
        
        update_metrics(metrics, payment, v[k], q[k], output, f'({k}, {advertisers[k]})')
    
    return metrics, previous_outputs[-1]
            
    

In [107]:
def single_allocation_without_replacement(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', np.copy(bids)
    metrics = init_metrics()
    selected_ads = np.zeros(bids.shape[0]) # keep track of ads that are selected in previous rounds of auction.
    
    if not dependent:
        q = rag_based_relevance(prompt, ads, )
    
    for t in range(num_of_segments):
        if dependent:
            q = rag_based_relevance(prompt + '\n' + curr, ads, )
        
        k, payment = randomized_selection(q, bids)
        
        assert selected_ads[k] == 0 # shouldn't have been selected before.
        # curr = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], curr_y=curr)
        # TODO: We ignore abobe line for now as we only care about metrics irrelevant to output.
        selected_ads[k] = 1 # k is winner
        update_metrics(metrics, payment, v[k], q[k], curr, f'({k}, {advertisers[k]})')
        bids[k] = 0 # never gonna be winner again
    
    return metrics, curr
            
    

In [108]:
def single_allocation_naive_i(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()
    
    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(q, bids)
        curr = curr + ads[k]
        update_metrics(metrics, payment, v[k], q[k], curr, f'({k}, {advertisers[k]})')
        
    return metrics, curr
    

In [109]:
def single_allocation_naive_ii(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()

    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(np.ones(bids.shape[0]), bids) # removing the effect of RAG -- second price auction
        # curr = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], curr_y=curr)
        # TODO: We ignore abobe line for now as we only care about metrics irrelevant to output.
        update_metrics(metrics, payment, v[k], q[k], curr, f'({k}, {advertisers[k]})')
        
    return metrics, curr
    

In [110]:
def get_relevency(y: str, A: list, ads: list):
    y_embedding = model.encode(y)
    ads_embedding = model.encode([ads[ad] for ad in A])
    return (1 + util.dot_score(y_embedding, ads_embedding)[0].numpy()) / 2
    

In [58]:
def multi_allocation_greedy(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    num_of_ads_in_each_segment: int):
    
    curr, v, n = '', bids, bids.shape[0]
    metrics = init_metrics()
    
    
    for t in range(num_of_segments):
        A = []
        
        while len(A) < num_of_ads_in_each_segment:
            adjusted_bids = np.zeros(n)
            for i in tqdm(range(n)):
                if i in A:
                    continue
            
                A_i = A + [i]
                y_A_i = segment_based_multi_ad_generation(prompt=prompt, advertisers=[advertisers[j] for j in A_i], ads=[ads[j] for j in A_i], curr_y=curr)
                
                q_A_i = get_relevency(y_A_i, A_i, ads)
                adjusted_bids[i] = np.dot(q_A_i, bids[np.array(A_i)])
                
            
            i_star = np.argmax(adjusted_bids)
            A = A + [i_star]
        
        y_A = segment_based_multi_ad_generation(prompt=prompt, advertisers=[advertisers[j] for j in A], ads=[ads[j] for j in A], curr_y=curr)
        q_A = get_relevency(y_A, A, ads)
        
        metrics[OUTPUT].append(y_A)
        metrics[SOCIAL_WELFARE].append(np.dot(q_A, bids[np.array(A)]))
        metrics[RELEVANCE].append(np.sum(q_A))
        curr = curr + '\n' + y_A
    
    return metrics, curr

In [21]:
prompt_for_adv = 'Give me a paragraph about {} so that I can use it to advertise this brand.'

## Sampling Bids and Running Auctions
We consider different scenarios in which we have a query $x$ and a set of advertisers. In different scenarios, we consider
+ different context for query;
+ different number of advertisers such that some of them are relevant, some are in the middle, and some others are irrelevant.

### Scenario 0

In [146]:
prompt = '"Can you suggest some books similar to \"To Kill a Mockingbird\"?"'
advertisers = ['Apple', 'Amazon', 'Costco', 'Starbucks',]

ads = []
for c in tqdm(advertisers):
    ads.append(query_to_chatgpt(prompt_for_adv.format(c)))

100%|██████████| 4/4 [00:19<00:00,  4.78s/it]


In [173]:
bids = np.array([3, 3, 2, 2,])
num_of_repeat = 100

In [174]:
get_allocation_vector(prompt, bids, ads, advertisers)

(q_i, b_i, adv_i)
(0.32, 3.00, Apple)
(0.45, 3.00, Amazon)
(0.32, 2.00, Costco)
(0.23, 2.00, Starbucks)
allocation vector:
[0.28271392 0.39585813 0.18597614 0.13545181]
x_max / x_min: 2.9225015493013573


In [176]:
import json
from tqdm import tqdm

In [188]:
runs = []
all_metrics = []

for N in tqdm(range(num_of_repeat)):
    metric, y = single_allocation_with_replacement(prompt=prompt, advertisers=advertisers, ads=ads, bids=bids, num_of_segments=3, dependent=False)
    
    all_metrics.append(metric)
    if N % 5 == 1:
        with open('scenario0_single_allocation_with_replacement.json', 'w') as f:
            f.write(json.dumps(all_metrics, indent=4))
    
    sw = np.sum(metric[SOCIAL_WELFARE])
    revenue = np.sum(metric[REVENUE])
    relevence = np.sum(metric[RELEVANCE])
    runs.append((sw, revenue, relevence,))

runs = np.array(runs)

vals = [[float(f'{np.mean(runs[:, j]):.3f}') for j in range(3)]]
print(f'Social Welfare: {np.mean(runs[:, 0]):.3f}\nRevenue: {np.mean(runs[:, 1]):.3f}\nRelevence: {np.mean(runs[:, 2]):.3f}')

100%|██████████| 100/100 [18:39<00:00, 11.20s/it]

Social Welfare: 2.989
Revenue: 0.971
Relevence: 1.089


In [189]:
ads

["Experience innovation and elegance with Apple, a global leader in technology that continually sets the standard for user-friendly interfaces, stunning designs, and groundbreaking advancements. From the iPhone's captivating visuals and camera technology to the seamless synchronization of the Apple ecosystem comprising iPads, MacBooks, and Apple Watches, Apple products are designed to enhance your life professionally and personally. Whether you're a creative professional, a student, or someone who enjoys cutting-edge technology, Apple offers a range of devices that combine sophisticated functionality with exclusive features like Face ID, Siri, and Apple Pay, all secured with industry-leading privacy protection. Embrace a world where technology meets simplicity and sophistication with Apple.",
 'Discover the vast and vibrant world of Amazon, your one-stop shop for everything from the latest technology and trendy fashion to everyday essentials and unique finds. With Amazon, experience se

In [150]:
query0 = f'''{prompt}\n please respond to this question for only one sentence while also advertise {advertisers[winners[0]]} with this context >>
{ads[winners[0]]}
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Make sure that response should be one sentence.
'''

In [158]:
from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model = "gpt-4-turbo",
  logprobs = False,
  temperature = 1,
  max_tokens = 300,
  messages=[
    {"role": "user", "content": query0,}
  ]
)

In [160]:
output0 = response.choices[0].message.content

In [161]:
output0

"If you appreciated the profound storytelling of “To Kill a Mockingbird,” consider reading “The Help” by Kathryn Stockett or “Go Set a Watchman” by Harper Lee during your quiet evenings, perfectly illuminated on the retina display of your iPad, an example of how Apple's technology blends sophistication with everyday practicality, enhancing your reading experience."

In [162]:
query1 = f'''
You must continue your answer to my original query.
Your previous response was
>> {output0}

And you now should advertise {advertisers[winners[1]]}, but without hurting the coherency of the entire document.
Here's some contexts about {advertisers[winners[1]]}

>>  {ads[winners[1]]}

Make sure that there is one new sentence.
Write the entire document, which merges your previous response and new paragraph.
'''

In [163]:
from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model = "gpt-4-turbo",
  logprobs = False,
  temperature = 1,
  max_tokens = 300,
  messages=[
    {"role": "user", "content": query0},
    {"role": "assistant", "content": output0},
    {"role": "user", "content": query1}
  ]
)

In [164]:
output1 = response.choices[0].message.content

In [165]:
output1

"If you appreciated the profound storytelling of “To Kill a Mockingbird,” consider reading “The Help” by Kathryn Stockett or “Go Set a Watchman” by Harper Lee during your quiet evenings, perfectly illuminated on the retina display of your iPad, an example of how Apple's technology blends sophistication with everyday practicality, enhancing your reading experience. Additionally, explore these titles and more on Amazon, where you can easily find both e-books and physical copies, ensuring a seamless shopping experience that complements your sophisticated Apple devices while taking advantage of Amazon Prime's myriad benefits, from free shipping to exclusive streaming content."

In [166]:
query2 = f'''
You must continue your answer to my original query.
Your previous response was
>> {output1}

And you now should advertise {advertisers[winners[2]]}, but without hurting the coherency of the entire document.
Here's some contexts about {advertisers[winners[2]]}

>>  {ads[winners[2]]}

Make sure that there is one new sentence.
Write the entire document, which merges your previous response and new paragraph.
'''

In [167]:
from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model = "gpt-4-turbo",
  logprobs = False,
  temperature = 1,
  max_tokens = 300,
  messages=[
    {"role": "user", "content": query0},
    {"role": "assistant", "content": output0},
    {"role": "user", "content": query1},
    {"role": "assistant", "content": output1},
    {"role": "user", "content": query2},
  ]
)

In [168]:
output2 = response.choices[0].message.content

In [169]:
output2

"If you appreciated the profound storytelling of “To Kill a Mockingbird,” consider reading “The Help” by Kathryn Stockett or “Go Set a Watchman” by Harper Lee during your quiet evenings, perfectly illuminated on the retina display of your iPad, an example of how Apple's technology blends sophistication with everyday practicality, enhancing your reading experience. Additionally, explore these titles and more on Amazon, where you can easily find both e-books and physical copies, ensuring a seamless shopping experience that complements your sophisticated Apple devices while taking advantage of Amazon Prime's myriad benefits, from free shipping to exclusive streaming content. And what better way to enjoy your new books than paired with a delicious, handcrafted beverage from Starbucks, enhancing each reading session with the perfect blend of comfort and taste, symbolizing a delightful union between literary enrichment and gourmet satisfaction."

In [170]:
ads

["Experience innovation and elegance with Apple, a global leader in technology that continually sets the standard for user-friendly interfaces, stunning designs, and groundbreaking advancements. From the iPhone's captivating visuals and camera technology to the seamless synchronization of the Apple ecosystem comprising iPads, MacBooks, and Apple Watches, Apple products are designed to enhance your life professionally and personally. Whether you're a creative professional, a student, or someone who enjoys cutting-edge technology, Apple offers a range of devices that combine sophisticated functionality with exclusive features like Face ID, Siri, and Apple Pay, all secured with industry-leading privacy protection. Embrace a world where technology meets simplicity and sophistication with Apple.",
 'Discover the vast and vibrant world of Amazon, your one-stop shop for everything from the latest technology and trendy fashion to everyday essentials and unique finds. With Amazon, experience se

### Scenario 1

prompt = `How to activate Internationl Roaming?`

advertisers = `['AT&T', 'Huawei', 'Samsung', 'LG', 'Costco', 'Starbucks',]`

In [19]:
prompt = 'How to activate Internationl Roaming?'
advertisers = ['AT&T', 'Huawei', 'Samsung', 'LG', 'Costco', 'Starbucks',]

ads = []
for c in tqdm(advertisers):
    ads.append(query_to_chatgpt(prompt_for_adv.format(c)))

('How to activate Internationl Roaming?',
 ['AT&T', 'Huawei', 'Samsung', 'LG', 'Costco', 'Starbucks'])

In [72]:
bids = np.array([3, 3, 2, 2, 1.5, 1.5])
num_of_repeat = 200

#### Single Allocation With Replacement

In [36]:
get_allocation_vector(prompt, bids, ads, advertisers)

(q_i, b_i, adv_i)
(0.56, 3.00, AT&T)
(0.40, 3.00, Huawei)
(0.46, 2.00, Samsung)
(0.57, 2.00, LG)
(0.38, 1.50, Costco)
(0.35, 1.50, Starbucks)
allocation vector:
[0.28063362 0.19765734 0.15161781 0.18957933 0.09454949 0.08596241]
x_max / x_min: 3.26460854975134


In [91]:
runs = []

for N in range(num_of_repeat):
    m, y = single_allocation_with_replacement(prompt=prompt, advertisers=advertisers, ads=ads, bids=bids, num_of_segments=3, dependent=False)
    sw = np.sum(m[SOCIAL_WELFARE])
    revenue = np.sum(m[REVENUE])
    relevence = np.sum(m[RELEVANCE])
    runs.append((sw, revenue, relevence,))

runs = np.array(runs)

vals = [[float(f'{np.mean(runs[:, j]):.3f}') for j in range(3)]]
print(f'Social Welfare: {np.mean(runs[:, 0]):.3f}\nRevenue: {np.mean(runs[:, 1]):.3f}\nRelevence: {np.mean(runs[:, 2]):.3f}')

Social Welfare: 3.504
Revenue: 0.571
Relevence: 1.431


In [92]:
runs = []

for N in range(num_of_repeat):
    m, y = single_allocation_without_replacement(prompt=prompt, advertisers=advertisers, ads=ads, bids=np.copy(bids), num_of_segments=3, dependent=False)
    sw = np.sum(m[SOCIAL_WELFARE])
    revenue = np.sum(m[REVENUE])
    relevence = np.sum(m[RELEVANCE])
    runs.append((sw, revenue, relevence,))

runs = np.array(runs)

vals.append([float(f'{np.mean(runs[:, j]):.3f}') for j in range(3)])
print(f'Social Welfare: {np.mean(runs[:, 0]):.3f}\nRevenue: {np.mean(runs[:, 1]):.3f}\nRelevence: {np.mean(runs[:, 2]):.3f}')

Social Welfare: 3.354
Revenue: 0.672
Relevence: 1.414


In [93]:
runs = []

for N in range(num_of_repeat):
    m, y = single_allocation_naive_ii(prompt=prompt, advertisers=advertisers, ads=ads, bids=np.copy(bids), num_of_segments=3, dependent=False)
    sw = np.sum(m[SOCIAL_WELFARE])
    revenue = np.sum(m[REVENUE])
    relevence = np.sum(m[RELEVANCE])
    runs.append((sw, revenue, relevence,))

runs = np.array(runs)

vals.append([float(f'{np.mean(runs[:, j]):.3f}') for j in range(3)])
print(f'Social Welfare: {np.mean(runs[:, 0]):.3f}\nRevenue: {np.mean(runs[:, 1]):.3f}\nRelevence: {np.mean(runs[:, 2]):.3f}')

Social Welfare: 3.269
Revenue: 0.247
Relevence: 1.389


In [76]:
import pandas as pd

In [85]:
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (ii)'],
    'Social Welfare': [3.558, 3.348, 3.267],
    'Revenue': [0.545, 0.674, 0.249],
    'Relevence': [1.444, 1.419, 1.383]
})

In [94]:
df # old

,Mechanism,Social Welfare,Revenue,Relevence
0,with Replacement,3.558,0.545,1.444
1,without Replacement,3.348,0.674,1.419
2,Naive (ii),3.267,0.249,1.383


In [95]:
vals = np.array(vals)

In [96]:
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (ii)'],
    'Social Welfare': vals[:, 0],
    'Revenue': vals[:, 1],
    'Relevence': vals[:, 2]
})

In [97]:
df

,Mechanism,Social Welfare,Revenue,Relevence
0,with Replacement,3.504,0.571,1.431
1,without Replacement,3.354,0.672,1.414
2,Naive (ii),3.269,0.247,1.389


In [113]:
y_orig = query_to_chatgpt(f'{prompt} please answer the question in exactly three paragraphs.')

In [114]:
paras = y_orig.split('\n\n')

'Finally, once international roaming is activated, make sure your phone is set up correctly to use the service abroad. This often involves enabling data roaming on your device through the network settings. Be cautious about data usage as roaming charges can be high. It’s also wise to switch off automatic software updates and any app updates to prevent background data usage. For added security and cost management, consider purchasing a local SIM card upon arrival or using Wi'

In [111]:
m, y = single_allocation_naive_ii(prompt=prompt, advertisers=advertisers, ads=ads, bids=np.copy(bids), num_of_segments=3, dependent=False)

In [122]:
m

{'social_welfare': [1.1915430128574371,
  0.9140016436576843,
  1.1915430128574371],
 'revenue': [0.22294871356860899, 0.13473650951779065, 0.15123860416570703],
 'relevance': [0.397181, 0.45700082, 0.397181],
 'output': ['', '', ''],
 'winner': ['(1, Huawei)', '(2, Samsung)', '(1, Huawei)']}

In [129]:
ys = []
for i in range(3):
    query = '''
    Rephrase this paragraph:\n"{}"\n while also advertise {} with this context:  "{}".
    Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
    Keep the answer short, there shoudln't be too much ad'''
    
    ad_ids = [1, 2, 1]
    ad_id = ad_ids[i]
    
    ys.append(query_to_chatgpt(query.format(paras[i], advertisers[ad_id], ads[ad_id],)))

In [125]:
ys[0]

'To enable international roaming for your upcoming travels, start by checking if your mobile service provider supports roaming in your destination country, typically accessible through their website under sections like "International" or "Travel Services." If supported, you can activate roaming conveniently online via your account or their mobile app, choosing from various plans that cater to different needs and trip durations. When selecting a device for travel, consider Huawei\'s range of products. Renowned for their robust technology and connectivity features, Huawei devices — from smartphones to wearables —\xa0ensure a seamless digital experience while abroad. Their reliable performance and long battery life make them an excellent choice for travelers needing consistent access to mobile services and superior technology.'

In [126]:
paras[0]

"To activate international roaming, you will first need to check if your current mobile service provider offers international roaming services for the country you plan to visit. Often, this information is readily available on the service provider's website under international or travel services. If available, you can usually activate it online by logging into your account or through the mobile service provider's app. Look for options like “Manage Services,” “International” or “Roaming” to select your desired roaming package. Some providers may have different plans based on the destination and length of stay, offering either pay-as-you-go rates or fixed-price packages."

In [130]:
paras[0] + '\n---------\n' + paras[1] + '\n---------\n' + paras[2]

"To activate international roaming, you will first need to check if your current mobile service provider offers international roaming services for the country you plan to visit. Often, this information is readily available on the service provider's website under international or travel services. If available, you can usually activate it online by logging into your account or through the mobile service provider's app. Look for options like “Manage Services,” “International” or “Roaming” to select your desired roaming package. Some providers may have different plans based on the destination and length of stay, offering either pay-as-you-go rates or fixed-price packages.\n---------\nIf you cannot activate international roaming online or prefer to do it through a customer service representative, you can call or visit a local branch of your service provider. When contacting them, make sure to inquire about the specific details of the roaming services, such as cost, data speeds, and coverage

In [131]:
ys[0] + '\n---------\n' + ys[1] + '\n---------\n' + ys[2]

"Before traveling internationally, it's important to ensure your mobile service provider supports international roaming in your destination country. Typically, you can find this information and activate roaming services directly through your provider's website or mobile app under sections like “Manage Services” or “Roaming.” Providers might offer different roaming plans based on the destination and duration of your stay, from pay-as-you-go to predetermined packages.\n\nAs you prepare for your travels, consider equipping yourself with Huawei's latest technology. Their devices, from smartphones and wearables to tablets, are crafted to deliver a seamless digital experience, ensuring you stay connected effortlessly, no matter where you roam. Huawei’s commitment to innovation and reliability means you can enjoy uninterrupted service and superior performance, enhancing your travel experience worldwide.\n---------\nIf you're unable to set up international roaming online, or you prefer a more 

In [134]:
y_init = query_to_chatgpt(prompt_to_init_answer.format(prompt, advertisers[1], ads[1]))

In [135]:
y_init

"To activate international roaming, usually, you first need to contact your mobile network provider or adjust settings in your mobile account to enable the service, which can often be done via an app or website. With a Huawei smartphone, this process becomes even more seamless and user-friendly, leveraging its intuitive user interface and robust connectivity features. Huawei devices, known for their reliability and cutting-edge technology, ensure that you remain connected in the most remote locations without missing a beat. This feature, combined with Huawei's commitment to high-quality, innovative technology, makes it an excellent choice for global travelers who need dependable communication solutions, ensuring you're always just a tap away from home."

In [ ]:
query = '''
Please continue your response
'''
y_second = query_to_chatgpt()

In [140]:
y_init

"To activate international roaming, usually, you first need to contact your mobile network provider or adjust settings in your mobile account to enable the service, which can often be done via an app or website. With a Huawei smartphone, this process becomes even more seamless and user-friendly, leveraging its intuitive user interface and robust connectivity features. Huawei devices, known for their reliability and cutting-edge technology, ensure that you remain connected in the most remote locations without missing a beat. This feature, combined with Huawei's commitment to high-quality, innovative technology, makes it an excellent choice for global travelers who need dependable communication solutions, ensuring you're always just a tap away from home."

In [144]:
from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model = "gpt-4-turbo",
  logprobs = False,
  temperature = 1,
  max_tokens = 300,
  messages=[
    {"role": "user", "content": prompt_to_init_answer.format(prompt, advertisers[1], ads[1])},
    {"role": "assistant", "content": y_init},
    {"role": "user", "content": f"Please continue your response for {prompt} for one more paragraph while minimally advertising {advertisers[2]} with context {ads[2]}. Don't put too much advertisement."}
  ]
)

In [145]:
response.choices[0].message.content

"Once international roaming is activated on your device, you can make the necessary adjustments to your phone settings to ensure optimal functionality while abroad. For example, on Samsung Galaxy devices, users can easily manage their data usage and select preferred network types through the 'Connections' section in the settings menu, which helps in controlling roaming charges and enhancing the user experience. Samsung's dedication to innovation is evident in these user-centric features that simplify technology management while you're on the move. Moreover, the sophisticated design and powerful capabilities of Samsung smartphones ensure that you stay connected and productive with style and efficiency, no matter where your travels take you."